<h1>
    Q.1 Modeling 
</h1>

<h3>Skill:</h3>
<ul>
    <li>Player 1 skill: $s_1$</li>
    <li>Player 2 skill: $s_2$</li>
</ul>
Where: $s_1 \sim \mathcal{N}(\mu_1,\,\sigma^{2}_1)$

Where: $s_2 \sim \mathcal{N}(\mu_2,\,\sigma^{2}_2)$

<h3>Game outcome:</h3>
<ul>
    <li>$t = s_1 - s_2$</li>
</ul>
Where: $t \sim \mathcal{N}(\mu_t = s_1 - s_2,\,\sigma^{2}_t)$

<h3>Game result:</h3>
<ul>
    <li>If player 1 wins: $y = 1$</li>
    <li>If player 2 wins: $y = -1$</li>
</ul>

<h3>Bayesian model:</h3>
<ul>
    <li>$P(s_1, s_2, t, y) = P(y|t) \cdot P(t|s_1, s_2) \cdot P(s_1) \cdot P(s_2)$</li>
</ul>
Where: 
<ul>
    <li>$P(y|t)$ Game result given game outcome</li>
    <li>$P(t|s_1, s_2)$ Game outcome given player skills</li>
    <li>$P(s_1)$ and $P(s_2)$ Player skill priors</li>
</ul>

<h3>Hyperparameters:</h3>
<ul>
    <li>Player 1: $\mu_1$ and $\sigma^{2}_1$</li>
    <li>Player 2: $\mu_2$ and $\sigma^{2}_2$</li>
    <li>Unpredictability of the game outcome: $\sigma^{2}_t$</li>
</ul>

<h1>
    Q.2 Bayesian Network
</h1>

$s_1$ and $s_2$ influences $t$ 

$t$ then influences $y$

<h1>
    Q.3 Computing with the model
</h1>

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.stats import truncnorm

<h3>Initialization</h3>

<h3>$p(s1, s2|t, y)$ - Full conditional distribution of the skills</h3>

In [2]:
# Gaussian-Gaussian Bayesian Update
def update_skills(s1_mean, s1_var, s2_mean, s2_var, t_var, t_samples):
    A = np.array([[1, -1]])
    mu_prior = np.array([[s1_mean], [s2_mean]])
    sigma_prior = np.array([[s1_var, 0], [0, s2_var]])
        
    sigma_posterior = np.linalg.inv(np.linalg.inv(sigma_prior) + (t_var)**(-1) * (A.T @ A))
    mu_posterior = sigma_posterior @ (np.linalg.inv(sigma_prior) @ mu_prior + t_var**(-1) * A.T * t_samples)

    return mu_posterior, sigma_posterior

mean_s1 = 1
var_s1 =  1
mean_s2 = -1
var_s2 = 4
var_t = 5
t_obs = 3

mu_posterior, Sigma_posterior = update_skills(mean_s1,var_s1,mean_s2,var_s2,var_t,t_obs)
print("Posterior mean of s1 and s2:", mu_posterior)
print("Posterior covariance of s1 and s2:", Sigma_posterior)

Posterior mean of s1 and s2: [[ 1.1]
 [-1.4]]
Posterior covariance of s1 and s2: [[0.9 0.4]
 [0.4 2.4]]


# <h3>$p(t|s1, s2, y)$ - Full conditional distribution of the outcome</h3>

In [3]:
def conditional_distribution_t(s1, s2, y):
    t_mean = s1 - s2
    a, b = 0, 0
    
    if y == 1:
        a = 0
        b = np.inf
    else:
        a = -np.inf
        b = 0
    return t_mean, a, b

s1 = 2
s2 = 1
t_var = 5
y = 1

mu, a, b = conditional_distribution_t(s1, s2, y)

print(f"Parameters for p(t|s1, s2, y):\nmu_t: {mu}, sigma^2: {round(t_var,4)} \nBounds: [{a}, {b}]")

Parameters for p(t|s1, s2, y):
mu_t: 1, sigma^2: 5 
Bounds: [0, inf]


<h3>$p(y=1)$ - Marginal probability that Player 1 wins the game</h3>

In [4]:
# Marginal Probability that Player 1 Wins
def compute_marginal_probability(mu_t, var_s1, var_s2, var_t):
    sigma_t = np.sqrt(var_s1 + var_s2 + var_t)
    probability_y1 = 1 - norm.cdf(0, mu_t, sigma_t)
    
    return probability_y1

var_s1 =  1
var_s2 =  4
mu_t = 2
t_var = 5

prob_y1 = compute_marginal_probability(mu_t, var_s1, var_s2, t_var)
print(f"Marginal probability that Player 1 wins (p(y=1)): {round(prob_y1,4)}")
print(f"Marginal probability that Player 2 wins (p(y=-1)): {round(1-prob_y1,4)}")


Marginal probability that Player 1 wins (p(y=1)): 0.7365
Marginal probability that Player 2 wins (p(y=-1)): 0.2635


<h1>
    Q.4  A first Gibbs sampler
</h1>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, truncnorm
import time

def gibbs_sampling_trueskill(N, N_burn_in, match_properties):
    s1_mean = match_properties["s1_mean"]
    s1_var = match_properties["s1_var"]
    s1_std = np.sqrt(s1_var)
    
    s2_mean = match_properties["s2_mean"]
    s2_var = match_properties["s2_var"]
    s2_std = np.sqrt(s2_var)


    t_var = match_properties["t_var"]
    
    y = match_properties["y"]

    
    s1_samples, s2_samples, t_samples = np.zeros(N), np.zeros(N), np.zeros(N)
    s1_samples[0] = (np.random.normal(s1_mean, s1_std))
    s2_samples[0] = (np.random.normal(s2_mean, s2_std))
    
    for i in range(N - 1):
        t_mean = s1_samples[i] - s2_samples[i]
        t_mean, a, b = conditional_distribution_t(s1_samples[i], s2_samples[i], y)
        t_samples[i+1] = truncnorm.rvs(a = (a-t_mean)/np.sqrt(t_var), b = (b-t_mean)/np.sqrt(t_var), loc=t_mean, scale=np.sqrt(t_var))
    
        mu_posterior, sigma_posterior = update_skills(s1_mean, s1_var, s2_mean, s2_var, t_var, t_samples[i]) 
    
        s1_samples[i+1], s2_samples[i+1] = np.random.multivariate_normal(mu_posterior.flatten(), sigma_posterior, 1).T
    
    return np.array(s1_samples[N_burn_in:]), np.array(s2_samples[N_burn_in:]), np.array(t_samples[N_burn_in:])
    
start_time = time.time()
s1_mean = 25
s1_var = (25/3)
s2_mean = 25
s2_var = (25/3)
t_var = (25/6)
y = 1

match_properties = {
    "s1_mean": s1_mean,
    "s1_var": s1_var,
    "s2_mean": s2_mean,
    "s2_var": s2_var,
    "t_var": t_var,
    "y": y
}


N = 1000
N_burn_in = int(np.sqrt(N))

s1_samples, s2_samples, t_samples = gibbs_sampling_trueskill(N, N_burn_in, match_properties)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

C:\Users\claud\AppData\Local\Temp\ipykernel_14848\890143259.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s1_samples[i+1], s2_samples[i+1] = np.random.multivariate_normal(mu_posterior.flatten(), sigma_posterior, 1).T


Elapsed time:  0.40168213844299316


In [ ]:
def plot_samples(s1_samples, s2_samples, t_samples):
    fig, ax = plt.subplots(1, 3, figsize=(12, 5))
    ax[0].plot(s1_samples, label="s1 samples")
    ax[0].set_title("s1 samples without burn-in")
    ax[0].legend()
    
    ax[1].plot(s2_samples, label="s2 samples")
    ax[1].set_title("s2 samples without burn-in")
    ax[1].legend()

    ax[2].plot(t_samples, label="t samples")
    ax[2].set_title("t samples without burn-in")
    ax[2].legend()
    
    plt.tight_layout()
    plt.show()

def plot_histogram_with_fitted_gaussian(samples, title, ax):
    mu = np.mean(samples)
    sigma = np.std(samples)
    ax.hist(samples, bins=2*int(np.sqrt(len(samples))), density=True, alpha=0.6, label="Samples after burn-in")
    ax.plot(np.linspace(min(samples), max(samples), 400), 
            norm.pdf(np.linspace(min(samples), max(samples), 400), mu, sigma), 
             label="Fitted Gaussian")
    ax.legend()
    ax.set_title(title)

plot_samples(s1_samples, s2_samples, t_samples)

fig, ax = plt.subplots(1, 3, figsize=(12, 3))
plot_histogram_with_fitted_gaussian(s1_samples, "Posterior of s1", ax[0])
plot_histogram_with_fitted_gaussian(s2_samples, "Posterior of s2", ax[0])
plot_histogram_with_fitted_gaussian(t_samples, "Posterior of t", ax[2])
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

def plot_histogram_with_fitted_gaussian(samples, title, ax, i, color="blue"):
    # Calculate mean and standard deviation
    mu = (25)
    sigma = ((25/3)**0.5)
    
    # Plot histogram with a better binning strategy and specified color
    n_bins = 2*int(np.sqrt(len(samples)))
    ax.hist(samples, bins=n_bins, density=True, alpha=0.6, color=color, label=f"Samples s{i}")
    
    # Plot the fitted Gaussian
    if i == 1:
        x_vals = np.linspace(10, 40, 400)
        ax.plot(x_vals, norm.pdf(x_vals, mu, sigma), color="red", label=f"Prior Gaussian: μ={mu:.2f}, σ={sigma:.2f}")
    
    # Enhance the visual appeal
    ax.set_title(title, fontsize=16)
    ax.legend(fontsize=12)
    ax.grid(True, which="both", linestyle="--", linewidth=0.5)
    ax.set_xlabel("Sample Value", fontsize=14)
    ax.set_ylabel("Density", fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)

fig, ax = plt.subplots(1, 1, figsize=(12, 5))
plot_histogram_with_fitted_gaussian(s1_samples, "", ax, 1, color="blue")
plot_histogram_with_fitted_gaussian(s2_samples, "Posterior of s1 and s2", ax, 2, color="green")
plt.tight_layout()
plt.savefig("post_n_samples.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3))
plot_histogram_with_fitted_gaussian(s1_samples, "1", ax)
plot_histogram_with_fitted_gaussian(s2_samples, "Posterior of s1 and s2, N = 10000", ax)

plt.savefig("post_n_10000.png")

In [ ]:
mu1 = np.mean(s1_samples)
mu1

In [ ]:
sigma1 = np.std(s1_samples)
sigma1

In [ ]:
mu2 = np.mean(s2_samples)
mu2

In [ ]:
sigma2 = np.std(s2_samples)
sigma2

In [ ]:
mu3 = np.mean(t_samples)
mu3

In [ ]:
sigma3 = np.var(t_samples[2000:])
sigma3

In [ ]:
plt.plot(s1_samples, label="S1")
plt.plot(s2_samples, label="S2")
plt.title("Plot samples")
plt.legend()
plt.show()

In [ ]:
plt.hist(s1_samples, bins = int(N**0.5), label="S1")
plt.hist(s2_samples, bins = int(N**0.5), label="S2")
plt.title("Histogram samples")
plt.legend()
plt.show()

In [ ]:
from scipy.stats import norm

def plot_normal_distributions(means, stds, labels=None, title="", xlabel="X", ylabel="Density"):
    """
    Plot multiple normal distributions overlaying each other.

    Parameters:
    - means: List of means for the normal distributions.
    - stds: List of standard deviations for the normal distributions.
    - labels: List of labels for the distributions.
    - title: The title of the plot.
    - xlabel: The label for the x-axis.
    - ylabel: The label for the y-axis.
    """
    
    # Create a range of x values to evaluate the distributions
    x = np.linspace(min(means) - 4 * max(stds), max(means) + 4 * max(stds), 1000)
    
    # If labels are not provided, create default labels
    if not labels:
        labels = [f"Dist {i}" for i in range(len(means))]
    
    # Plot each distribution
    for mean, std, label in zip(means, stds, labels):
        y = norm.pdf(x, mean, std)
        plt.plot(x, y, label=label)
    
    # Set the title, labels, and display the legend
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    plt.show()

# Sample distributions
sample_means = [mu1, mu2, 25]
sample_stds = [sigma1, sigma2, (25/3)]
sample_labels = ["Dist A", "Dist B", "Dist C"]

# Plot the sample distributions
plot_normal_distributions(sample_means, sample_stds, sample_labels)


<h1>
    Q.5  Assumed Density Filtering
</h1>

In [ ]:
import pandas as pd

seriea_df = pd.read_csv('SerieA.csv')

# Extract all unique teams
teams = list(set(seriea_df['team1'].unique()) | set(seriea_df['team2'].unique()))

# Initialize each team's skills with a common prior
initial_mean = 25
initial_variance = (25/3)**2

team_skills = {team: {'mean': initial_mean, 'variance': initial_variance} for team in teams}
team_skills


In [ ]:
N = 250
N_burn_in = 10

for index, row in seriea_df.iterrows():
    team1, team2, score1, score2 = row['team1'], row['team2'], row['score1'], row['score2']
    
    # Skip the match if it's a draw
    if score1 == score2:
        continue
        
    # Determine the outcome (1 if team1 wins, -1 if team2 wins)
    y = 1 if score1 > score2 else -1
    
    # Get the current skill estimates for the two teams
    s1_mean, s1_var = team_skills[team1]['mean'], team_skills[team1]['variance']
    s2_mean, s2_var = team_skills[team2]['mean'], team_skills[team2]['variance']

    t_var = (25/6)
    
    match_properties = {
        "s1_mean": s1_mean,
        "s1_var": s1_var,
        "s2_mean": s2_mean,
        "s2_var": s2_var,
        "t_var": t_var,
        "y": y
    }

    # Use the Gibbs sampler to get posterior skill distributions
    s1_samples, s2_samples, t_samples = gibbs_sampling_trueskill(N, N_burn_in, match_properties)
    
    # Update the skill estimates for the two teams
    team_skills[team1]['mean'], team_skills[team1]['variance'] = np.mean(s1_samples), np.var(s1_samples)
    team_skills[team2]['mean'], team_skills[team2]['variance'] = np.mean(s2_samples), np.var(s2_samples)

# Sort teams by their final skill means to get the ranking
sorted_teams = sorted(team_skills.items(), key=lambda x: x[1]['mean'], reverse=True)

ranked_teams = [(team[0], team[1]['mean']) for team in sorted_teams]
ranked_teams

In [ ]:
plt.figure(figsize=(12, 8))

for team in team_skills.keys():
    mean = team_skills[team]['mean']
    std_dev = np.sqrt(team_skills[team]['variance'])
    x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)
    plt.plot(x, norm.pdf(x, mean, std_dev), label=team)

plt.title('Skill Levels of All Teams')
plt.xlabel('Skill Level')
plt.ylabel('Probability Density')
plt.legend()
plt.grid()
plt.show()

<h1>
    Q.6 Using the model for predictions
</h1>

In [ ]:
N = 1000
N_burn_in = 100

match_predictions = []

# Prediction function
def predict_winner(s1_mean, s1_var, s2_mean, s2_var):
    diff_mean = s1_mean - s2_mean
    diff_var = s1_var + s2_var
    # Compute the probability that Player 1's skill is greater than Player 2's skill
    return 1 if compute_marginal_probability(diff_mean, s1_var, s2_var, diff_var) > 0.5 else -1

correct_predictions = 0
total_predictions = 0

for index, row in seriea_df.iterrows():
    team1, team2, score1, score2 = row['team1'], row['team2'], row['score1'], row['score2']

    # Skip the match if it's a draw
    if score1 == score2:
        continue

    # Determine the outcome (1 if team1 wins, -1 if team2 wins)
    y = 1 if score1 > score2 else -1
    
    # Predict the winner based on current skill estimates
    prediction = predict_winner(
        team_skills[team1]['mean'], team_skills[team1]['variance'],
        team_skills[team2]['mean'], team_skills[team2]['variance']
    )

    # Append match details and prediction to the list
    match_predictions.append({
        "team1": team1,
        "team2": team2,
        "actual_outcome": y,
        "predicted_outcome": prediction
    })
    
    
    # Compare prediction to actual result
    if prediction == y:
        correct_predictions += 1

    total_predictions += 1
    
    # Get the current skill estimates for the two teams
    s1_mean, s1_var = team_skills[team1]['mean'], team_skills[team1]['variance']
    s2_mean, s2_var = team_skills[team2]['mean'], team_skills[team2]['variance']

    t_var = 1
    
    match_properties = {
        "s1_mean": s1_mean,
        "s1_var": s1_var,
        "s2_mean": s2_mean,
        "s2_var": s2_var,
        "t_var": t_var,
        "y": y
    }

    # Use the Gibbs sampler to get posterior skill distributions
    s1_samples, s2_samples, t_samples = gibbs_sampling_trueskill(N, N_burn_in, match_properties)
    
    # Update the skill estimates for the two teams
    team_skills[team1]['mean'], team_skills[team1]['variance'] = np.mean(s1_samples), np.var(s1_samples)
    team_skills[team2]['mean'], team_skills[team2]['variance'] = np.mean(s2_samples), np.var(s2_samples)

# Compute prediction rate
prediction_rate = correct_predictions / total_predictions

prediction_rate

<h1>
    Q.9 Your own data
</h1>

In [29]:
import pandas as pd

# Read the CSV file into a DataFrame
df_cs = pd.read_csv('results.csv')

# Convert the 'date' column to datetime format
df_cs['date'] = pd.to_datetime(df_cs['date'])

# Define the year for filtering
year = 2016

# Filter the data for the specified year's H1 and H2
df_2016_h1 = df_cs.loc[(df_cs['date'].dt.year == year) & df_cs['date'].dt.month.isin(range(1, 7))]
df_2016_h2 = df_cs.loc[(df_cs['date'].dt.year == year) & df_cs['date'].dt.month.isin(range(7, 13))]

# Extract the specified columns for both datasets
cols = ['team_1', 'team_2', 'result_1', 'result_2']
df_2016_h1 = df_2016_h1[cols]
df_2016_h2 = df_2016_h2[cols]

# Drop rows with question marks for both datasets
df_2016_h1 = df_2016_h1.loc[~((df_2016_h1['team_1'] == '?') | (df_2016_h1['team_2'] == '?'))]
df_2016_h2 = df_2016_h2.loc[~((df_2016_h2['team_1'] == '?') | (df_2016_h2['team_2'] == '?'))]

# Extract unique teams 
teams_h1 = set(df_2016_h1['team_1'].unique()) | set(df_2016_h1['team_2'].unique())
teams_h2 = set(df_2016_h2['team_1'].unique()) | set(df_2016_h2['team_2'].unique())

# Combine the unique teams from both halves
teams = list(teams_h1 | teams_h2)

df_2016_h1 = df_2016_h1.reset_index(drop=True)
df_2016_h2 = df_2016_h2.reset_index(drop=True)


In [30]:
# Extract all unique teams
teams = list(teams)

# Initialize each team's skills with a common prior
initial_mean = 25
initial_variance = (25/3)**2
team_skills = {team: {'mean': initial_mean, 'variance': initial_variance} for team in teams}

In [31]:
team_skills

{'Incept': {'mean': 25, 'variance': 69.44444444444446},
 'HEADSHOTBG': {'mean': 25, 'variance': 69.44444444444446},
 'ex-31337': {'mean': 25, 'variance': 69.44444444444446},
 'OnlineBOTS': {'mean': 25, 'variance': 69.44444444444446},
 'Dark Sided': {'mean': 25, 'variance': 69.44444444444446},
 'DeToNator': {'mean': 25, 'variance': 69.44444444444446},
 'FLuffy Gangsters': {'mean': 25, 'variance': 69.44444444444446},
 'Noble': {'mean': 25, 'variance': 69.44444444444446},
 'NRG': {'mean': 25, 'variance': 69.44444444444446},
 'LGR': {'mean': 25, 'variance': 69.44444444444446},
 'seven': {'mean': 25, 'variance': 69.44444444444446},
 'ZZCY': {'mean': 25, 'variance': 69.44444444444446},
 'Mineski': {'mean': 25, 'variance': 69.44444444444446},
 'GODSENT': {'mean': 25, 'variance': 69.44444444444446},
 'AVANT': {'mean': 25, 'variance': 69.44444444444446},
 'Innova': {'mean': 25, 'variance': 69.44444444444446},
 'inchk1ng': {'mean': 25, 'variance': 69.44444444444446},
 'autocracy': {'mean': 25, '

In [32]:
N = 10
N_burn_in = 1

for index, row in df_2016_h1.iterrows():
    team1, team2, score1, score2 = row['team_1'], row['team_2'], row['result_1'], row['result_2']
    if (index % 25 == 0):
        print(index)
        
    # Determine the outcome (1 if team1 wins, -1 if team2 wins)
    y = 1 if score1 > score2 else -1
    
    # Get the current skill estimates for the two teams
    s1_mean, s1_var = team_skills[team1]['mean'], team_skills[team1]['variance']
    s2_mean, s2_var = team_skills[team2]['mean'], team_skills[team2]['variance']

    t_var = 25/6
    
    match_properties = {
        "s1_mean": s1_mean,
        "s1_var": s1_var,
        "s2_mean": s2_mean,
        "s2_var": s2_var,
        "t_var": t_var,
        "y": y
    }

    # Use the Gibbs sampler to get posterior skill distributions
    s1_samples, s2_samples, t_samples = gibbs_sampling_trueskill(N, N_burn_in, match_properties)
    
    # Update the skill estimates for the two teams
    team_skills[team1]['mean'], team_skills[team1]['variance'] = np.mean(s1_samples), np.var(s1_samples)
    team_skills[team2]['mean'], team_skills[team2]['variance'] = np.mean(s2_samples), np.var(s2_samples)

# Sort teams by their final skill means to get the ranking
sorted_teams = sorted(team_skills.items(), key=lambda x: x[1]['mean'], reverse=True)

ranked_teams = [(team[0], team[1]['mean']) for team in sorted_teams]
ranked_teams

0
25
50


C:\Users\claud\AppData\Local\Temp\ipykernel_14848\890143259.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s1_samples[i+1], s2_samples[i+1] = np.random.multivariate_normal(mu_posterior.flatten(), sigma_posterior, 1).T


75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950


[('ELTZ', 28.82212420763133),
 ('FTW. Prinfor', 28.740864341671404),
 ('TYLOO', 28.011544126022685),
 ('demise', 27.886915496910152),
 ('TeamOne', 27.822076935152253),
 ('Bravado', 27.43283963845027),
 ('eGeneration', 26.580143375506378),
 ('VG.CyberZen', 26.303239844970363),
 ('MeMyself&I', 26.213602603605448),
 ('ex-CPH Wolves', 26.210767482642435),
 ('fnatic', 25.974862552631624),
 ('TheMongolz', 25.91744870626206),
 ('Natus Vincere', 25.899433093557725),
 ('mousesports', 25.89681968206168),
 ('WeGotGame', 25.72733231535695),
 ('nEophyte', 25.611819725639712),
 ('Immunity', 25.559427314980862),
 ('Envy', 25.452126604623103),
 ('Virtus.pro', 25.18395401115795),
 ('Complexity', 25.103625465857792),
 ('Millenium', 25.081153806618325),
 ('Deathtrap', 25.01482448937297),
 ('Incept', 25),
 ('HEADSHOTBG', 25),
 ('ex-31337', 25),
 ('OnlineBOTS', 25),
 ('Dark Sided', 25),
 ('ZZCY', 25),
 ('AVANT', 25),
 ('autocracy', 25),
 ('TRS', 25),
 ('VenatoreS', 25),
 ('iNTRO', 25),
 ('VwS', 25),
 ('Muf

In [ ]:
N = 100
N_burn_in = 10

match_predictions = []

# Prediction function
def predict_winner(s1_mean, s1_var, s2_mean, s2_var):
    diff_mean = s1_mean - s2_mean
    diff_var = s1_var + s2_var
    # Compute the probability that Player 1's skill is greater than Player 2's skill
    return 1 if compute_marginal_probability(diff_mean, s1_var, s2_var, diff_var) > 0.5 else -1

correct_predictions = 0
total_predictions = 0

for index, row in df_2016_h2.iterrows():
    team1, team2, score1, score2 = row['team_1'], row['team_2'], row['result_1'], row['result_2']

    # Skip the match if it's a draw
    if score1 == score2:
        continue

    # Determine the outcome (1 if team1 wins, -1 if team2 wins)
    y = 1 if score1 > score2 else -1
    
    # Predict the winner based on current skill estimates
    prediction = predict_winner(
        team_skills[team1]['mean'], team_skills[team1]['variance'],
        team_skills[team2]['mean'], team_skills[team2]['variance']
    )

    # Append match details and prediction to the list
    match_predictions.append({
        "team1": team1,
        "team2": team2,
        "actual_outcome": y,
        "predicted_outcome": prediction
    })
    
    
    # Compare prediction to actual result
    if prediction == y:
        correct_predictions += 1

    total_predictions += 1
    
    # Get the current skill estimates for the two teams
    s1_mean, s1_var = team_skills[team1]['mean'], team_skills[team1]['variance']
    s2_mean, s2_var = team_skills[team2]['mean'], team_skills[team2]['variance']

    t_var = 25/6
    
    match_properties = {
        "s1_mean": s1_mean,
        "s1_var": s1_var,
        "s2_mean": s2_mean,
        "s2_var": s2_var,
        "t_var": t_var,
        "y": y
    }

    # Use the Gibbs sampler to get posterior skill distributions
    s1_samples, s2_samples, t_samples = gibbs_sampling_trueskill(N, N_burn_in, match_properties)
    
    # Update the skill estimates for the two teams
    team_skills[team1]['mean'], team_skills[team1]['variance'] = np.mean(s1_samples), np.var(s1_samples)
    team_skills[team2]['mean'], team_skills[team2]['variance'] = np.mean(s2_samples), np.var(s2_samples)

# Compute prediction rate
prediction_rate = correct_predictions / total_predictions

prediction_rate

C:\Users\claud\AppData\Local\Temp\ipykernel_14848\890143259.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  s1_samples[i+1], s2_samples[i+1] = np.random.multivariate_normal(mu_posterior.flatten(), sigma_posterior, 1).T


In [ ]:
df_2016_h2

<h1>Q10 - Include draws</h1>

In [ ]:
import pandas as pd

seriea_df = pd.read_csv('SerieA.csv')

# Extract all unique teams
teams = list(set(seriea_df['team1'].unique()) | set(seriea_df['team2'].unique()))

# Initialize each team's skills with a common prior
initial_mean = 25
initial_variance = (25/3)**2

team_skills = {team: {'mean': initial_mean, 'variance': initial_variance} for team in teams}
team_skills


In [ ]:
N = 250
N_burn_in = 10

for index, row in seriea_df.iterrows():
    team1, team2, score1, score2 = row['team1'], row['team2'], row['score1'], row['score2']
    
    # The changes --------------------
    if score1 > score2:
        y = 1
    elif score1 == score2:
        if np.abs(team_skills[team1]['mean'] - team_skills[team2]['mean']) > 2:
            if team_skills[team1]['mean'] > team_skills[team2]['mean']:
                y = -1
            else:
                y = 1
            
    else:
        y = -1
            
        
        
    # Determine the outcome (1 if team1 wins, -1 if team2 wins)
    y = 1 if score1 > score2 else -1
    
    # Get the current skill estimates for the two teams
    s1_mean, s1_var = team_skills[team1]['mean'], team_skills[team1]['variance']
    s2_mean, s2_var = team_skills[team2]['mean'], team_skills[team2]['variance']

    t_var = (25/6)
    
    match_properties = {
        "s1_mean": s1_mean,
        "s1_var": s1_var,
        "s2_mean": s2_mean,
        "s2_var": s2_var,
        "t_var": t_var,
        "y": y
    }

    # Use the Gibbs sampler to get posterior skill distributions
    s1_samples, s2_samples, t_samples = gibbs_sampling_trueskill(N, N_burn_in, match_properties)
    
    # Update the skill estimates for the two teams
    team_skills[team1]['mean'], team_skills[team1]['variance'] = np.mean(s1_samples), np.var(s1_samples)
    team_skills[team2]['mean'], team_skills[team2]['variance'] = np.mean(s2_samples), np.var(s2_samples)

# Sort teams by their final skill means to get the ranking
sorted_teams = sorted(team_skills.items(), key=lambda x: x[1]['mean'], reverse=True)

ranked_teams = [(team[0], team[1]['mean']) for team in sorted_teams]
ranked_teams

In [ ]:
N = 100
N_burn_in = 10

match_predictions = []

# Prediction function
def predict_winner(s1_mean, s1_var, s2_mean, s2_var):
    diff_mean = s1_mean - s2_mean
    diff_var = s1_var + s2_var
    # Compute the probability that Player 1's skill is greater than Player 2's skill
    if compute_marginal_probability(diff_mean, s1_var, s2_var, diff_var) > 0.50:
        return 1
    else:
        return -1

correct_predictions = 0
total_predictions = 0

for index, row in seriea_df.iterrows():
    team1, team2, score1, score2 = row['team1'], row['team2'], row['score1'], row['score2']

    # Check the values
    if score1 > score2:
        y = 1
    elif score1 == score2:
        if np.abs(team_skills[team1]['mean'] - team_skills[team2]['mean']) > 2:
            if team_skills[team1]['mean'] > team_skills[team2]['mean']:
                y = -1
            else:
                y = 1
            
    else:
        y = -1
    
    # Predict the winner based on current skill estimates
    prediction = predict_winner(
        team_skills[team1]['mean'], team_skills[team1]['variance'],
        team_skills[team2]['mean'], team_skills[team2]['variance']
    )

    # Append match details and prediction to the list
    match_predictions.append({
        "team1": team1,
        "team2": team2,
        "actual_outcome": y,
        "predicted_outcome": prediction
    })
    
    
    # Compare prediction to actual result
    if prediction == y:
        correct_predictions += 1

    total_predictions += 1
    
    # Get the current skill estimates for the two teams
    s1_mean, s1_var = team_skills[team1]['mean'], team_skills[team1]['variance']
    s2_mean, s2_var = team_skills[team2]['mean'], team_skills[team2]['variance']

    t_var = 1
    
    match_properties = {
        "s1_mean": s1_mean,
        "s1_var": s1_var,
        "s2_mean": s2_mean,
        "s2_var": s2_var,
        "t_var": t_var,
        "y": y
    }

    # Use the Gibbs sampler to get posterior skill distributions
    s1_samples, s2_samples, t_samples = gibbs_sampling_trueskill(N, N_burn_in, match_properties)
    
    # Update the skill estimates for the two teams
    team_skills[team1]['mean'], team_skills[team1]['variance'] = np.mean(s1_samples), np.var(s1_samples)
    team_skills[team2]['mean'], team_skills[team2]['variance'] = np.mean(s2_samples), np.var(s2_samples)

# Compute prediction rate
prediction_rate = correct_predictions / total_predictions

prediction_rate